In [1]:
import argparse

from tqdm import tqdm

In [2]:
%load_ext autoreload
%autoreload 2

from load_data import load_data
from model import GModel
from optimizer import Optimizer
from myutils import *
from sampler import NewSampler
from sklearn.model_selection import KFold

In [4]:
for data in ['nci', 'gdsc1', 'gdsc2', 'ctrp']:

    class Args:
        def __init__(self):
            self.device = torch.device(
                "cuda" if torch.cuda.is_available() else "cpu"
            )  # cuda:number or cpu
            self.data = data
    
    args = Args()
    
    res, drug_feature, exprs, mut, cna, null_mask = load_data(args)
    cell_sum = np.sum(res, axis=1)
    drug_sum = np.sum(res, axis=0)
    for target_dim in [0, 1]:
        print('cell' if target_dim == 0 else 'drug')
        
        samples = res.shape[target_dim]
        
        # フィルタ処理
        passed_targets = []
        skipped_targets = []
        
        for target_index in range(samples):
            label_vec = res.iloc[target_index] if target_dim == 0 else res.iloc[:, target_index]
            passed, reason, pos, neg, total = filter_target(label_vec)
        
            if passed:
                passed_targets.append(target_index)
            else:
                skipped_targets.append((target_index, reason, pos, neg, total))
        
        # スキップ対象を表示
        print(f"\n🚫 Skipped Targets: {len(skipped_targets)}")
        for idx, reason, pos, neg, total in skipped_targets:
            print(f"Target {idx}: skipped because {reason} (total={total}, pos={pos}, neg={neg})")

load nci
response matrix (res) shape: (977, 59)
exprs shape: (60, 23059)
mut shape: (60, 9307)
cna shape: (60, 23232)
59
exprs shape: (59, 23059)
mut shape: (59, 9307)
cna shape: (59, 23232)
drug_feature shape: (976, 920)
response matrix (res) shape: (59, 976)
null_mask shape: (59, 976)
cell

🚫 Skipped Targets: 2
Target 15: skipped because low_positive_ratio (total=488, pos=7, neg=481)
Target 33: skipped because low_negative_ratio (total=427, pos=419, neg=8)
drug

🚫 Skipped Targets: 123
Target 27: skipped because few_total_samples (total=3, pos=2, neg=1)
Target 28: skipped because few_total_samples (total=9, pos=5, neg=4)
Target 49: skipped because few_total_samples (total=5, pos=2, neg=3)
Target 57: skipped because few_total_samples (total=9, pos=1, neg=8)
Target 105: skipped because few_total_samples (total=9, pos=2, neg=7)
Target 106: skipped because few_total_samples (total=9, pos=3, neg=6)
Target 111: skipped because few_total_samples (total=7, pos=2, neg=5)
Target 114: skipped be

In [ ]:
python run_nihgcn.py --data nci --target drug && \
python run_nihgcn.py --data gdsc1 --target drug && \
python run_nihgcn.py --data gdsc1 --target cell && \
python run_nihgcn.py --data gdsc2 --target drug && \
python run_nihgcn.py --data gdsc2 --target cell && \
python run_nihgcn.py --data ctrp --target drug && \
python run_nihgcn.py --data ctrp --target cell